<a href="https://colab.research.google.com/github/jh941213/SequenceData-DL-/blob/main/Numpy%E1%84%85%E1%85%A9_RNN_%E1%84%86%E1%85%A1%E1%86%AB%E1%84%83%E1%85%B3%E1%86%AF%E1%84%80%E1%85%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [4]:
random_generator = np.random.default_rng() # data set

def generate_data(n_features, n_values): #피처맵과 벨류) n_values = 데이터의 피처수, 열의 수
    features = random_generator.random((n_features, n_values)) 
    targets = random_generator.random((n_features)) # 타겟 생성
    return features, targets # 피처와 타겟을 반환
    #target = np.random.choice([0,1]) #이진분류를 위한

In [21]:
trainX, trainY = generate_data(5000, 4) # 5000개의 행과 4개의 열
testX, testY = generate_data(5, 4)

In [22]:
trainX.shape

(5000, 4)

In [16]:
trainY.shape

(500,)

In [23]:
class RNN:
    def __init__(self): # 모든 변수를 초기화 
        self.global_weight = [1, 1] # [Input, Recurrent Weight]
        self.local_weight = [0.001, 0.001] #0.0001에 왜 2개일까 입력이 2개 기 때문이다 이전시점 t, x
        self.W_sign = [0, 0] # 부정적인 예측의 경우 순환 신경망의 벌 

        self.eta_p = 1.2 # 예상시간을 찾는데 사용된다 eta_P, eta_n
        self.eta_n = 0.5
# x, state 를 수신하는 역할을 해준다.
    def state_handler(self, input_x, previous_state):
        return input_x * self.global_weight[0] + previous_state * self.global_weight[1]

#순방향 전파 : 신경망 하나가 신경망 끝으로 가는 그과정이다
    def forward_propagation(self, X):
      # Computes the forward propagation of the RNN.
      S = np.zeros((X.shape[0], X.shape[1]+1))
      for k in range(0, X.shape[1]):
          next_state = self.state_handler(X[:,k], S[:,k]) #실제 입력 마지막 state입력과 + state 를 결합
          S[:,k+1] = next_state
      return S
#역방향
    def backward_propagation(self, X, S, grad_out):
      # Computes the backward propagation of the RNN.
        grad_over_time = np.zeros(( X.shape[0], X.shape[1]+1 ))
        grad_over_time[:,-1] = grad_out

        wx_grad = 0
        wy_grad = 0
        for k in range(X.shape[1], 0, -1):
            wx_grad += np.sum( grad_over_time[:, k] * X[:, k-1] ) # 2차원의 데이터의 경우 wx, wy를 변경해야 한다
            wy_grad += np.sum( grad_over_time[:, k] * S[:, k-1] )

            grad_over_time[:, k-1] = grad_over_time[:, k] * self.global_weight[1]
        return (wx_grad, wy_grad), grad_over_time
#업데이트 함수 : 앞으로 함수를 호출한 다음 뒤로 함수를 호출한다.(grad_out 손실)
    def update_rprop(self, X, Y, W_prev_sign, local_weight):
        S = self.forward_propagation(X)
        grad_out = 2 * (S[:, -1] - Y) / 500
        W_grads, _ = self.backward_propagation(X, S, grad_out)
        self.W_sign = np.sign(W_grads) # 기울기를 반전하는 데 사용 모델을 좀더 빠르게
        # 글로벌 가중치
        for i, _ in enumerate(self.global_weight):
            if self.W_sign[i] == W_prev_sign[i]: #글로벌 가중치와 w.sign 이 같다면 eta_n을 변경해야한다.
                local_weight[i] *= self.eta_p
            else:
                local_weight[i] *= self.eta_n
        self.local_weight = local_weight
#최종함수로 x, y epochs d에 전달 , 에폭을 반복해주고 
    def train(self, X, Y, training_epochs):
        for epochs in range(training_epochs):
            self.update_rprop(X, Y, self.W_sign, self.local_weight)

            for i, _ in enumerate(self.global_weight):
                self.global_weight[i] -= self.W_sign[i] * self.local_weight[i]

In [24]:
rnn = RNN()
rnn.train(trainX, trainY, 200)

print (f"Targets are: {testY}")
y = rnn.forward_propagation(testX)[:, -1]
print (f"Predicted are: {y}")

Targets are: [0.93336367 0.15346702 0.74083594 0.58666673 0.25704375]
Predicted are: [0.44817772 0.4723367  0.76989868 0.33150121 0.56443057]
